In [38]:
import numpy as np


class MyNN:

    def __init__(self, l, q, d,
                 etall=.5, eta1=None, eta2=None,
                 ):
        self.eta1 = eta1 if eta1 is not None else etall
        self.eta2 = eta2 if eta2 is not None else etall

        self.l = l
        self.q = q
        self.d = d

        self.V = np.random.rand(d, q)
        self.W = np.random.rand(q, l)
        self.THETA = np.random.rand(l)
        self.GAMMA = np.random.rand(q)

        self.errs = list()

    def __f(self, X):
        return 1 / (1 + np.exp(-X))

    def estimate(self, X):
        X = np.asarray(X)
        ALPHA = np.matmul(X, self.V)
        B = self.__f(ALPHA - self.GAMMA)
        BETA = np.matmul(B, self.W)
        Y = self.__f(BETA-self.THETA)
        return Y

    def ek(self, X, Y):
        YB = self.estimate(X)

        Y = np.asarray(Y)
        YYB = Y - YB
        err = np.matmul(YYB, YYB)/2.0
        return err

    def eks(self, D):
        n = len(D)
        err = 0.0
        for X, Y in D:
            err += self.ek(X, Y)
        return err / n

    def bp(self, X, Y):

        X = np.asarray(X)
        Y = np.asarray(Y)
        e1 = self.eta1
        e2 = self.eta2

        ALPHA = np.matmul(X, self.V)
        B = self.__f(ALPHA - self.GAMMA)
        BETA = np.matmul(B, self.W)
        YB = self.__f(BETA-self.THETA)

        YYB = Y - YB
        err = np.matmul(YYB, YYB)/2.0
        self.errs.append(err)
        # print(err)

        G = YB * (1 - YB) * YYB
        E = B * (1 - B) * np.matmul(self.W, self.THETA)

        G1 = e1*G
        E2 = e2*E

        self.THETA -= - G1
        self.W += np.outer(B, G1)

        self.GAMMA = self.GAMMA - E2
        self.V = self.V + np.outer(X, E2)


In [39]:
import numpy as np


class MyDNN:

    def __init__(self, Q, etall=.5, ETA=None):
        if len(Q) < 3:
            raise ValueError
        self.Q = Q
        self.m = len(Q) - 2

        if ETA is None:
            self.ETA = [etall,] * (self.m+1)
        elif len(ETA) != self.m+1:
            raise ValueError
        else:
            self.ETA = ETA

        self.W = [0,]
        self.THETA = [0,]
        for c in range(1, self.m+2):
            self.W.append(np.random.rand(Q[c-1], Q[c]))
            self.THETA.append(np.random.rand(Q[c]))

        self.errs = list()

    def __f(self, X):
        return 1 / (1 + np.exp(-X))

    def estimate(self, X):
        B = np.asarray(X)
        for c in range(1, self.m+2):
            B = self.__f(np.matmul(B, self.W[c]) - self.THETA[c])
        return B

    def ek(self, X, Y):
        YK = self.estimate(X)
        YYK = YK - np.asarray(Y)
        err = np.matmul(YYK, YYK)/2.0
        return err

    def bp(self, X, Y):
        B = [0,] * (self.m+2)
        Z = [0,] * (self.m+2)
        G = [0,] * (self.m+2)
        ETA = [0,] + list(self.ETA)

        # b^{(0)} = x
        B[0] = np.asarray(X)

        for c in range(1, self.m+2):
            B[c] = self.__f(np.matmul(B[c-1], self.W[c]) - self.THETA[c])

        YK = B[self.m+1]
        YYB = YK - np.asarray(Y)
        err = np.matmul(YYB, YYB) / 2.0
        self.errs.append(err)
        # print(err)

        Z[self.m+1] = YYB

        for c in range(self.m+1, 0, -1):
            G[c] = Z[c] * B[c] * (B[c] - 1)
            Z[c-1] = -np.matmul(self.W[c], G[c])
            GC = ETA[c] * G[c]
            self.THETA[c] -= GC
            self.W[c] += np.outer(B[c-1], G[c])


In [40]:
import sklearn
from sklearn import datasets
import matplotlib.pyplot as plt


In [41]:
digits = datasets.load_digits()
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))


In [42]:
n_epochs = 100
n0 = 1024
n1 = n_samples - n0
l = 10
q = 16
d = 64
Q1 = (d, q, l)
Q2 = (d, q, q, l)


In [43]:
def test():
    vv = 1
    for i in range(n0, n_samples):
        if np.argmax(nn.estimate(data[i])) == digits.target[i]:
            vv += 1
    return vv / n1


nn = MyNN(l, q, d, etall=0.005)
# nn = MyDNN(Q1, etall=0.005)
rate = list()
errs = list()

flg = 0
while flg < n_epochs:
    for i in range(n_samples):
        a = np.zeros(10)
        a[digits.target[i]] = 1
        noise = np.random.random_sample(64) / 10.0
        nn.bp(data[i] + noise, a)
    vv = test()
    rate.append(vv)
    err = np.average(nn.errs)
    errs.append(err)
    nn.errs = list()

    print(vv, err)

    flg += 1


0.10090556274256145 4.486161370731173
0.10090556274256145 4.468631836955303
0.10090556274256145 4.074538623514703
0.10090556274256145 3.420755393118369
0.10090556274256145 3.2781704060998136
0.10090556274256145 3.2741270184458133
0.10090556274256145 3.1367061131516327
0.10090556274256145 2.8785400828388794
0.10090556274256145 2.877417421636188
0.10090556274256145 2.8770576986014156
0.10090556274256145 2.8765967933043415
0.10090556274256145 2.8758918422274546
0.10090556274256145 2.8745987879037336
0.10090556274256145 2.8710267807457663
0.10090556274256145 2.74608927577069
0.10090556274256145 2.4748742981537104
0.10090556274256145 2.4735466222659985
0.10090556274256145 2.473161670733689
0.10090556274256145 2.472697655192749
0.10090556274256145 2.472057451465204
0.10090556274256145 2.471098994673887
0.10090556274256145 2.469475111686454
0.10090556274256145 2.465972177338339
0.10090556274256145 2.4468737457055667
0.10090556274256145 1.9459734548177363
0.10090556274256145 1.6663806959567848

In [44]:
plt.plot(range(len(errs)), errs, label='Ek')
plt.title('Rate')
plt.legend()
plt.grid()
plt.savefig('./ek.png')
plt.clf()

plt.plot(range(len(rate)), rate, label='Rate')
plt.title('Ek')
plt.legend()
plt.grid()
plt.savefig('./rate.png')
plt.clf()


<Figure size 640x480 with 0 Axes>

In [45]:
nn.estimate(data[0])

array([0.09814689, 0.1010267 , 0.0977884 , 0.1013551 , 0.10146678,
       0.10095078, 0.10072945, 0.09955263, 0.09629831, 0.10029908])